# Classical vs. Quantum Accuracy: Technical Interpretation

This experiment evaluates two fundamentally different computational paradigms—classical machine learning and variational quantum circuits—on the same ASL hand-landmark dataset. The comparison highlights how model capacity, feature dimensionality, and representational structure directly affect downstream classification performance.

## 1. Classical SYN Baseline Model  
The classical pipeline uses the full 63-dimensional MediaPipe landmark vector, capturing joint positions across all fingers and the wrist. A Random Forest classifier (or equivalent classical architecture) is well-suited for this type of structured geometric input because:
- It retains all 63 input degrees of freedom with no information compression.
- It can model non-linear interactions between landmark triples (e.g., fingertip curvature, inter-joint distances).
- It exploits ensemble decision boundaries that are expressive enough to separate fine-grained hand shapes.

The resulting test accuracy is approximately 82%, reflecting the fact that the classical model is operating directly on high-dimensional, information-rich features with sufficient representational capacity.

## 2. Variational Quantum Circuit (VQC) Model  
The quantum model operates under far stricter constraints. A four-qubit variational circuit can only directly encode four independent continuous parameters via single-qubit rotations. To map the original 63-dimensional landmarks into a 4-dimensional quantum-compatible representation, PCA is applied. This compression step discards the overwhelming majority of geometric information present in the original landmarks.

Additionally, a 4-qubit VQC with two parameterized layers has extremely limited expressive power:
- Its hypothesis class is orders of magnitude smaller than that of a classical ensemble model.
- The circuit can only represent a narrow family of unitary transformations.
- Entangling operations across four qubits cannot reconstruct the high-order geometry lost during dimensionality reduction.

Consequently, the quantum model achieves an accuracy of approximately 3.6%, which is statistically indistinguishable from random guessing when multiple classes are present.

## 3. Explanation for the Discrepancy  
The outcome is driven by three structural factors:

1. **Feature Bottleneck:**  
   Reducing 63 dimensions to 4 removes most of the discriminative variation needed to distinguish ASL gestures. Classical models operate on the full representation; the quantum model must operate on a severely compressed embedding.

2. **Limited Quantum Expressivity:**  
   Variational circuits with a small number of qubits and shallow depth cannot model the nonlinear landmark interactions inherent in sign-language recognition. Classical models scale their capacity with tree depth or number of estimators; quantum models cannot scale similarly without increasing qubits or depth, both of which are computationally prohibitive in simulation.

3. **Simulation Constraints:**  
   The experiment is performed on a classical machine simulating quantum circuits. These simulations do not confer quantum advantages and are computationally expensive, making it impractical to increase depth or qubit count.

## 4. Interpretation  
The experiment does not imply that quantum machine learning is ineffective. Rather, it demonstrates the current limitations of small-scale quantum circuits for high-dimensional geometric classification. VQCs perform best on low-dimensional, algebraically structured problems (e.g., optimization, sampling, or small-qubit classification tasks). In contrast, ASL recognition is inherently a high-dimensional, spatially structured vision task that favors classical architectures.

## 5. Final Summary  
- The classical SYN model benefits from full-information input and high model capacity, achieving approximately 82% accuracy.  
- The quantum VQC model, constrained by a 4-qubit architecture and forced dimensionality reduction, achieves approximately 3.6% accuracy.  
- The performance gap reflects representational limitations of small-scale quantum models rather than a flaw in the dataset or experimental setup.

This comparison accurately illustrates the current boundary between classical and quantum learning systems when applied to real-world recognition tasks.


# Future Directions and Possible Improvements

While the current experiment deliberately constrains the quantum model to a small four-qubit architecture and a heavily compressed feature space, there are several technically meaningful avenues for improvement. These can be grouped into three broad categories: data representation, model architecture, and evaluation strategy.

## 1. Data and Feature Representation

1. **Task-specific dimensionality reduction instead of plain PCA**  
   PCA is purely variance-preserving and is agnostic to class labels. It may discard low-variance but highly discriminative directions. Replacing PCA with supervised or manifold-aware methods (e.g., Linear Discriminant Analysis, supervised autoencoders, or contrastive representation learning) could yield a 4–8 dimensional embedding that is more aligned with class separation and therefore more amenable to quantum encoding.

2. **Local, structured embeddings of the hand geometry**  
   Instead of flattening the entire hand into a single vector, one can construct features describing relative distances, angles, and normalized bone lengths. Compressing these higher-level features to a small dimension before quantum encoding may preserve more semantic structure per qubit than raw landmark coordinates.

3. **Class subset experiments**  
   Focusing on a smaller, carefully chosen subset of signs (e.g., letters with similar global hand shapes but subtle local differences) would provide a more controlled benchmark for quantum models and allow the exploration of whether quantum kernels or variational circuits can capture fine-grained distinctions given sufficient representational tuning.

## 2. Quantum and Hybrid Model Architecture

1. **Deeper and wider variational circuits**  
   Increasing the number of layers and exploring richer entanglement patterns (e.g., hardware-efficient ansätze, block-entangled architectures, or problem-inspired ansätze) would expand the function class represented by the circuit. This comes with an increased simulation cost but may reveal performance gains on smaller or subsampled datasets.

2. **More qubits with structured encoding**  
   Moving from 4 to 6–8 qubits enables higher-dimensional feature embeddings and potentially more expressive quantum decision boundaries. To mitigate the simulation cost, one could explore qubit pruning, parameter sharing, or tensor-network-based simulators designed for shallow circuits.

3. **Hybrid quantum–classical networks**  
   Rather than using a single linear head after the quantum layer, one can treat the quantum circuit as a feature extractor feeding into a small classical neural network. Jointly training the quantum and classical components may recover some of the expressive power lost through dimensionality reduction while still keeping the quantum component interpretable.

4. **Quantum kernel methods**  
   An alternative to variational circuits is to use fixed feature maps and quantum kernels evaluated via a quantum device or simulator. Kernel-based methods (e.g., quantum support vector machines) may provide more stable training dynamics and clearer theoretical guarantees on expressivity for certain feature maps.

## 3. Training, Regularization, and Evaluation

1. **Mini-batch optimization and learning rate schedules**  
   The current training loop uses full-batch gradient descent with a fixed step size. Introducing mini-batch optimization, adaptive learning rate schedules, and early stopping criteria may improve convergence quality and reduce overfitting to the compressed feature space.

2. **Systematic hyperparameter search**  
   The number of layers, entangling pattern, initialization scale, batch size, and learning rate were chosen heuristically. A systematic hyperparameter search—or Bayesian optimization over circuit and optimizer parameters—could identify configurations that exploit the limited quantum capacity more effectively.

3. **Robustness and calibration analysis**  
   Beyond accuracy, one could study calibration (e.g., reliability diagrams), decision margins, and robustness to noise or perturbations in the landmarks. This might reveal regimes where quantum models behave qualitatively differently from classical counterparts even if overall accuracy remains lower.

## 4. Towards Hardware-Relevant Demonstrations

1. **Noise-aware simulation and realistic hardware constraints**  
   Introducing physically motivated noise channels (dephasing, amplitude damping, readout errors) into the simulation would shift the study from an idealized setting to one that more closely approximates near-term quantum devices. This can guide the design of noise-resilient ansätze and error mitigation strategies.

2. **Reduced-scale prototypes for actual hardware**  
   Once a compact, low-qubit configuration with clear structure is identified (for example, 3–4 qubits on a binary or ternary subtask), it could be deployed on a real quantum processor. Comparing simulated vs. hardware performance would provide empirical insight into the practical feasibility of quantum sign recognition components.

Collectively, these directions move the project from a proof-of-concept comparison—highlighting the limitations of a small, compressed variational circuit—toward a more systematic exploration of how quantum models might be integrated as specialized components within larger classical systems for gesture or sign language recognition.


In [1]:
from google.colab import files
uploaded = files.upload()   # upload asl_landmarks_labeled.csv here

import pandas as pd
import numpy as np

df = pd.read_csv("asl_landmarks_labeled.csv")
print("Loaded:", df.shape)
df.head()

Saving asl_landmarks_labeled.csv to asl_landmarks_labeled.csv
Loaded: (2800, 64)


,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,label
0,0.303408,0.597586,-5.838650e-07,0.439908,0.548425,-0.039604,0.524477,0.405544,-0.041700,0.545018,...,0.188302,0.316252,-0.082758,0.202406,0.405406,-0.073136,0.216944,0.470087,-0.046558,A
1,0.288708,0.536942,-3.487557e-07,0.361038,0.504086,-0.016503,0.407451,0.424711,-0.016682,0.412333,...,0.277712,0.332662,-0.029891,0.278550,0.389221,-0.024470,0.276375,0.434777,-0.011344,A
2,0.710648,0.884992,-4.474464e-07,0.798471,0.844863,-0.022967,0.862417,0.760463,-0.028151,0.871160,...,0.663414,0.682990,-0.065655,0.672172,0.742343,-0.058456,0.675503,0.793246,-0.041162,A
3,0.711866,0.752371,-5.115814e-07,0.828867,0.652734,-0.042542,0.893890,0.499236,-0.056166,0.892567,...,0.603413,0.450060,-0.110367,0.632790,0.546571,-0.096903,0.639947,0.613654,-0.066352,A
4,0.456254,0.587702,-6.443583e-07,0.570339,0.504806,-0.035954,0.639931,0.373736,-0.044998,0.651392,...,0.393831,0.322019,-0.070336,0.407674,0.406351,-0.055679,0.405752,0.460163,-0.028924,A


In [2]:
from sklearn.preprocessing import LabelEncoder

# Features = all 63 landmark columns
X = df.drop("label", axis=1).values.astype("float32")   # shape (N, 63)
y_text = df["label"].values

# Convert letters (A, B, C...) → numbers (0,1,2...)
le = LabelEncoder()
y = le.fit_transform(y_text).astype("int64")

print("X shape:", X.shape)
print("y unique classes:", len(np.unique(y)))

X shape: (2800, 63)
y unique classes: 28


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# Scale X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# PCA down to 4 features → 4 qubits
pca = PCA(n_components=4)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

# Scale PCA results to [-pi/2, pi/2] for quantum rotations
mm = MinMaxScaler(feature_range=(-np.pi/2, np.pi/2))
X_train_q = mm.fit_transform(X_train_pca)
X_test_q  = mm.transform(X_test_pca)

print("Quantum feature shapes:", X_train_q.shape, X_test_q.shape)

Quantum feature shapes: (2240, 4) (560, 4)


In [4]:
!pip install -q pennylane

import pennylane as qml
from pennylane import numpy as pnp

n_qubits = 4
num_classes = len(np.unique(y))
n_layers = 2

dev = qml.device("default.qubit", wires=n_qubits)

# ----- Quantum Circuit -----
def encode_features(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

def variational_block(weights):
    for l in range(weights.shape[0]):
        for q in range(n_qubits):
            qml.RX(weights[l, q, 0], wires=q)
            qml.RY(weights[l, q, 1], wires=q)
            qml.RZ(weights[l, q, 2], wires=q)
        for q in range(n_qubits - 1):
            qml.CNOT(wires=[q, q+1])
        qml.CNOT(wires=[n_qubits-1, 0])

@qml.qnode(dev)
def qnode(x, weights):
    encode_features(x)
    variational_block(weights)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# ----- Parameters -----
weights = pnp.random.normal(0, 0.1, (n_layers, n_qubits, 3), requires_grad=True)
W_head = pnp.random.normal(0, 0.1, (n_qubits, num_classes), requires_grad=True)
b_head = pnp.zeros((num_classes,), requires_grad=True)

# ----- Helper Functions -----
def quantum_features(x_batch, weights):
    return pnp.stack([qnode(x, weights) for x in x_batch])

def forward(x_batch, weights, W_head, b_head):
    q_feats = quantum_features(x_batch, weights)
    logits = q_feats @ W_head + b_head
    return logits

def cross_entropy_loss(logits, y_true):
    exps = pnp.exp(logits - pnp.max(logits, axis=1, keepdims=True))
    probs = exps / pnp.sum(exps, axis=1, keepdims=True)
    idx = pnp.arange(len(y_true))
    chosen = probs[idx, y_true]
    return -pnp.mean(pnp.log(chosen + 1e-9))

def accuracy(logits, y_true):
    preds = pnp.argmax(logits, axis=1)
    return pnp.mean((preds == y_true).astype(float))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 50.6 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [5]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)

X_train_q_pl = pnp.array(X_train_q, requires_grad=False)
y_train_pl   = pnp.array(y_train, dtype=int)
X_test_q_pl  = pnp.array(X_test_q, requires_grad=False)
y_test_pl    = pnp.array(y_test, dtype=int)

params = [weights, W_head, b_head]
epochs = 10

for epoch in range(epochs):
    def cost_fn(*params):
        logits = forward(X_train_q_pl, *params)
        return cross_entropy_loss(logits, y_train_pl)

    params = opt.step(cost_fn, *params)
    weights, W_head, b_head = params

    train_logits = forward(X_train_q_pl, weights, W_head, b_head)
    test_logits  = forward(X_test_q_pl, weights, W_head, b_head)

    train_acc = accuracy(train_logits, y_train_pl)
    test_acc  = accuracy(test_logits, y_test_pl)

    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.3f} | Test Acc = {test_acc:.3f}")

quantum_acc = float(test_acc)
print("\nFINAL QUANTUM ACCURACY:", quantum_acc)

Epoch 1: Train Acc = 0.036 | Test Acc = 0.045
Epoch 2: Train Acc = 0.036 | Test Acc = 0.048
Epoch 3: Train Acc = 0.037 | Test Acc = 0.050
Epoch 4: Train Acc = 0.037 | Test Acc = 0.052
Epoch 5: Train Acc = 0.036 | Test Acc = 0.048
Epoch 6: Train Acc = 0.037 | Test Acc = 0.041
Epoch 7: Train Acc = 0.036 | Test Acc = 0.039
Epoch 8: Train Acc = 0.035 | Test Acc = 0.036
Epoch 9: Train Acc = 0.034 | Test Acc = 0.036
Epoch 10: Train Acc = 0.034 | Test Acc = 0.036

FINAL QUANTUM ACCURACY: 0.03571428571428571


In [6]:
import pandas as pd

df_compare = pd.DataFrame([
    {"Model": "SYN classical", "Accuracy": 0.82},
    {"Model": "Quantum VQC", "Accuracy": quantum_acc}
])
df_compare

,Model,Accuracy
0,SYN classical,0.820000
1,Quantum VQC,0.035714
